In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
from sklearn.utils import resample

In [5]:
def tabla_ganancia_individual(piso,pend,tope,max_correccion):
    '''Se le pasa los parametros y 
    calcula segun la cantidad corregida cuanto ganaria.
    Devuelve el dataframe con los valores para cada cantidad de correcciones'''
    
    max_correccion = max_correccion*10
    df = pd.DataFrame({'cantidad_examenes':np.arange(0,max_correccion+1,1)})
    df['funcion'] = 0*len(df)
    
    for ex in range(int(max_correccion+1)):
        if ex == 0:
            df.loc[ex,'funcion'] = piso
        
        elif df.loc[ex-1,'funcion'] >= tope:
             df.loc[ex:,'funcion'] = tope 
             break
        else:
            df.loc[ex,'funcion'] = df.loc[ex-1,'funcion'] + pend
        if df.loc[ex,'funcion'] >=tope:
            df.loc[ex,'funcion'] = tope
    return(df)


################################################################################


def costo_indiv_concurso(df, cantidad_ex):
    '''Para cada corrector devuelve lo que hay que pagarle por concurso
       segun la cantidad de examenes que corrige'''
    costo_indiv = df.loc[df.cantidad_examenes==int(cantidad_ex),'funcion'].values[0]
#     print('costo_corrector: ',costo_indiv)
    return costo_indiv


################################################################################

def tabla_costo_anual(correctores_inicial,piso,pend,tope,max_correccion,año,agregan,divide_en_max):
    '''DEVUELVE:
        [0] : dataframe de costos para toda la distribucion (original o bootstrapp). Es todo un año.
        [1] : suma de costos total para toda la dist (por ej todo el año)
        [2] : df de los pagos individuales segun los parametros (lo que sale de calcular_lineal_max)'''
    
    # TABLA DE COSTOS INDIVIDUALES PARA ESTOS PARAMETROS
    tabla_ganancia = tabla_ganancia_individual(piso,pend,tope,max_correccion)
    
    data_concursos = {}

    '''ERROR -> ESTABA SOBREESCRIBIENDO PORQUE ESTOY USANDO UN VALOR COMO INDICE'''
    for ID,conc in enumerate(año): # PARA CADA CONCURSO

        # CALCULO LA CANTIDAD DE CORRECTORES PARA CADA CONCURSO Y LA CANT DE EXAMENES Q TIENEN Q CORREGIR
        data_concursos[ID] = []
        data_concursos[ID].append(conc)

        '''la division es de a pares, no es completa'''
        if conc <= max_correccion:
            correctores = correctores_inicial
            cant_cada_uno = conc//(correctores//2)
            cant_cada_uno = max_correccion if cant_cada_uno > max_correccion else cant_cada_uno
        #hasta 2max hacemos que cada examen sea corregido por minimo dos. Si son 3 iniciales pagamos mas. 
        #a menos que divide_en_max == 1
        elif (conc <= 2*max_correccion):
            correctores = correctores_inicial
            '''divide_en_max==1 significa que empezamos a dividir cuando pasamos el primer maximo, no el segundo'''
            cant_cada_uno = conc//correctores if (divide_en_max==1) else conc//(correctores//2)
            cant_cada_uno = max_correccion if cant_cada_uno > max_correccion else cant_cada_uno
            
        #despues, se divide todo por la cantidad de correctores
        else:
            # esto va asi porq empezamos a sumar correctores cuando pasa EL SEGUNDO MAXIMO
            cant = conc - max_correccion
            correctores = correctores_inicial
            while cant > max_correccion:
                cant = cant - max_correccion
                correctores += agregan
            cant_cada_uno = conc//correctores        
            cant_cada_uno = max_correccion if cant_cada_uno > max_correccion else cant_cada_uno

        #CALCULO COSTO INDIVIDUAL DE CORRECTOR EN UN CONCURSO (SEGUN EXAMENES)
        costo_corrector = costo_indiv_concurso(tabla_ganancia,cant_cada_uno)
        
        #MULTIPLICO Y CALCULO EL COSTO DEL CONCURSO
        costo_concurso = correctores * costo_corrector

        # ARMO LO NECESARIO
        data_concursos[ID].append(correctores)
        data_concursos[ID].append(cant_cada_uno)
        data_concursos[ID].append(costo_corrector)
        data_concursos[ID].append(costo_concurso)        

        # INCLUYO EN EL DF
        data_concursos_df = pd.DataFrame.from_dict(data_concursos,orient='index').reset_index()
        data_concursos_df.columns= ['ID','EXAMENES_por_concurso','correctores','exam_cada_uno','ganancia_cada_corrector','costo_total']

        
    return (data_concursos_df,data_concursos_df.costo_total.sum(),tabla_ganancia)


In [6]:
def simulaciones_año(simulaciones,correctores_inicial,piso,pend,tope,max_correccion,año,agregan,divide_en_max,variacion):
    '''para una combinacion de parametros y cantidad de concursos para el año, 
       simula UN AÑO, varias veces'''
    
    costos = []

    # CANTIDAD DE AÑOS SIMULADOS PARA ESTA COMBINACION DE PARAMETROS
    for i in range(simulaciones):
        print(f'Simulacion nro: {i}')

        # CANTIDAD DE CONCURSOS        
        cant_concursos = int( variacion * len(año) )
                
        #SIMULO UN AÑO
        año_simulado = resample(año, replace=True, n_samples= cant_concursos)

        #CALCULO EL COSTO DEL AÑO SIMULADO (SOLO EL RESULTADO)
        costo_año = tabla_costo_anual(correctores_inicial,piso,pend,tope,max_correccion,año_simulado,agregan,divide_en_max)[1]

        #LO AGREGO A LISTA
        costos.append(costo_año)

    return np.array(costos)

################################################################################

def armado_original(params, año20):
    '''PARA EL AÑO 2020 CON EL NUEVO ESQUEMA'''

    df_original = params.copy()

    #CALCULO EL COSTO TOTAL DEL AÑO
    df_original['costo_nuevo'] = df_original.apply(lambda x:
             tabla_costo_anual(x.correctores_inicial,x.piso,x.pend,x.tope,
             x.max_correccion,dist20,x.agregan,x.divide_en_max)[1],axis='columns') # el [1] devuelve solo el costo total


    #CALCULO LA DIFERENCIA CON EL LIMITE (SI DA NEGATIVO ES QUE NOS PASAMOS)
    df_original['diferencia_limite'] = df_original.limite - df_original.costo_nuevo

    #CALCULO CUAL ES EL PORCENTAJE POR EL QUE NOS PASAMOS (SI DA NEGATIVO ES QUE NOS PASAMOS)
    df_original['cambio_porcen_limite'] =  df_original.diferencia_limite / df_original.limite

    #COSTO VIEJO
    df_original['costo_viejo'] = 2*31200*len(año20)

    #DIFERENCIA CON EL METODO VIEJO (SI DA NEGATIVO ES QUE NOS PASAMOS)
    df_original['diferencia_metodo'] = df_original.costo_viejo - df_original.costo_nuevo

    return df_original

################################################################################

def armado_simulacion(params, año, simulaciones):
    
    # BOOTSTRAP
    
    metodo_cant_concursos = {'FIJO20':1, 'MITAD_MAS':1.5,'DOBLE':2}
    
    for cant_concursos,multiplicar_por in metodo_cant_concursos.items():
        
        print('\nCANTIDAD DE CONCURSOS (ESCALADO): '+cant_concursos)
        
        '''LISTA DE COSTOS DE LOS AÑOS SIMULADOS'''
        params[f'{cant_concursos}_costo_bootstrap'] = params.apply(lambda x:
                simulaciones_año(simulaciones, x.correctores_inicial,x.piso,x.pend,x.tope,
                                     x.max_correccion,año,x.agregan,x.divide_en_max,
                                     multiplicar_por),axis='columns')
        
        '''COSTO MEDIO DE LOS AÑOS SIMULADOS'''
        params[f'{cant_concursos}_media_costo'] = params.apply(lambda x: x[f'{cant_concursos}_costo_bootstrap'].mean(), axis='columns')
        
        '''LISTA DE DIFERENCIAS DE LOS COSTOS CON EL LIMITE (SI DA NEGATIVO ES QUE NOS PASAMOS)'''
        params[f'{cant_concursos}_diferencia_limite_boot'] = params.apply(lambda x: np.array([x.limite - costo for costo in x[f'{cant_concursos}_costo_bootstrap']]), axis='columns')
        
        '''DIFERENCIA MEDIA Y STD DE LOS COSTOS CON EL LIMITE (SI DA NEGATIVO ES QUE NOS PASAMOS)'''
        params[f'{cant_concursos}_media_boot'] = params.apply(lambda x: x[f'{cant_concursos}_diferencia_limite_boot'].mean(), axis='columns')
        params[f'{cant_concursos}_std_boot'] = params.apply(lambda x: x[f'{cant_concursos}_diferencia_limite_boot'].std(), axis='columns')
        
        '''PORCENTAJE DE AÑOS SIMULADOS QUE NOS PASAMOS DEL LIMITE'''
        params[f'{cant_concursos}_porcentaje_inferior_0'] = params.apply(lambda x: len(x[f'{cant_concursos}_diferencia_limite_boot'][x[f'{cant_concursos}_diferencia_limite_boot']<0])/
                                                                 len(x[f'{cant_concursos}_diferencia_limite_boot']),axis='columns')
    
    return(params)


################################################################################

def plotear(tipo='violin',piso=6000,tope=40000,max_correccion=100,divide_en_max=1,correctores_inicial=2,agregan=1):
    data_plot = RESULTADOS[(RESULTADOS.piso==piso) & 
                       (RESULTADOS.tope==tope) & 
                       (RESULTADOS.max_correccion==max_correccion) & 
                       (RESULTADOS.divide_en_max==divide_en_max) &
                       (RESULTADOS.correctores_inicial==correctores_inicial) &
                       (RESULTADOS.agregan==agregan)]
    
    print(f'PENDIENTE: ${data_plot.pend.values[0]} por exámen')
    print(f'LÍMITE PRESUPUESTARIO: ${limite}')
    


    # PLOT VIOLIN
    if tipo=='violin':
        df_plot = pd.DataFrame({'FIJO20':data_plot['FIJO20_costo_bootstrap'].values[0],
                                'MITAD_MAS':data_plot['MITAD_MAS_costo_bootstrap'].values[0],
                                'DOBLE':data_plot['DOBLE_costo_bootstrap'].values[0]})
        df_plot = df_plot.melt()
        df_plot.columns = ['cantidad_concursos','costo']
        fig = px.violin(df_plot, y="costo", x="cantidad_concursos", box=True, color='cantidad_concursos')
        fig.update_layout(shapes=[
            dict(
              type= 'line',
              xref= 'paper', x0= 0, x1= 1,
              yref= 'y', y0= limite, y1= limite )])
        fig.show()
    
    # PLOT HISTOGRAM
    else:
        fig,axes = plt.subplots(3,1,figsize=(14,11),sharex=True)
        
        sns.distplot(data_plot.FIJO20_costo_bootstrap.values[0],ax=axes[0])
        axes[0].axvline(x=limite,color='red',linestyle='--')
        axes[0].set_title(f'Simulaciones con misma cantidad de concursos que 2020 (22). PROMEDIO de COSTO: ${round(data_plot.FIJO20_media_costo.values[0],1)}')

        sns.distplot(data_plot.MITAD_MAS_costo_bootstrap.values[0],ax=axes[1])
        axes[1].axvline(x=limite,color='red',linestyle='--')
        axes[1].set_title(f'Simulaciones con %50 más de concursos que 2020 (33). PROMEDIO de COSTO: ${round(data_plot.MITAD_MAS_media_costo.values[0],1)}')

        sns.distplot(data_plot.DOBLE_costo_bootstrap.values[0],ax=axes[2])
        plt.axvline(x=limite,color='red',linestyle='--')
        axes[2].set_title(f'Simulaciones con el doble de concursos que 2020 (44). PROMEDIO de COSTO: ${round(data_plot.DOBLE_media_costo.values[0],1)}')
        plt.text(x=limite,y=0,s=f'límite presupuesto: ${limite}',color='red',fontsize=14)
        plt.xlabel('COSTO')
        
        
    # EJEMPLO
    from IPython.display import display
    pend= round((tope-piso)/(max_exam),0)
    tabla_mostrar = tabla_costo_anual(correctores_inicial,piso,pend,tope,
                      max_correccion,dist_año,agregan,divide_en_max)[0].drop(columns='ID')
    display(tabla_mostrar.sort_values('EXAMENES_por_concurso',ascending=True))
    print('Costo para el año 2020 con estos parametros: $',tabla_mostrar.costo_total.sum())
    
    
def plotear_resultado(RESULTADOS, *vec_params):
    slider = widgets.interact(plotear,
                    tipo = ['violin','hist'],
                    piso = piso_vec,
                    tope = tope_vec,
                    max_correccion = max_correccion_vec,
                    divide_en_max = divide_en_max_vec,
                    correctores_inicial=correctores_inicial_vec,
                    agregan = agregan_vec)

    display(slider)

In [7]:
'''Repaso
Comienza con correctores_inicial y se reparten asegurando que todos los examenes los corrijan por lo menos dos. 
Y cuando llega a (1 si divide_en_max==1 else 2) * max_correccion, se agrega la cantidad agrega.'''

'''LIMITE PRESUPUESTARIO'''
# limite = (3_000_000 - (3_000_000*0.16))
limite = 3_950_000

'''DISTRIBUCION PRUEBA'''

# primera
# dist_año = pd.Series([17,170,227,773,585,431,710,741,337,266,240,597,236,87,
#                      257,196,212,167,78,391,143,45,1500,1000,615])

# año 2020
#dist_año = pd.Series([17,170,227,773,585,431,710,741,337,266,240,597,236,87,257,196,
#                      212,167,78,391,143,45])

# año 2021
dist_año = pd.Series([322,850,667,491,809,826,375,306,290,673,292,112,301,255,237,270,120,572,213,64])


correctores_inicial = 2
piso=6000
tope=45000
max_exam=100
porcentaje_hasta_tope= 1
pend= round((tope-piso)/(max_exam*porcentaje_hasta_tope),0)
# pend=200
agregan=1
divide_en_max=1
print('Piso: $',piso)
print('Tope: $',tope)
print('Maximo examen por corrector: ',max_exam)
print('Pendiente: $',pend)

results, total, df = tabla_costo_anual(correctores_inicial,piso,pend,tope,
                                     max_exam,dist_año,agregan,divide_en_max)

results['costo_por_examen'] = results.costo_total / results.EXAMENES_por_concurso
print('COSTO TOTAL del año: $',results.costo_total.sum())
print(f'\nPresupuesto: ${limite} - Costo nuevo: ${results.costo_total.sum()} \nRESULTADO= ${limite-results.costo_total.sum()} \nRESULTADO(%)= %{round((limite-results.costo_total.sum())/limite*100,2)}')
results.sort_values('EXAMENES_por_concurso')

Piso: $ 6000
Tope: $ 45000
Maximo examen por corrector:  100
Pendiente: $ 390.0
COSTO TOTAL del año: $ 3700470.0

Presupuesto: $3950000 - Costo nuevo: $3700470.0 
RESULTADO= $249530.0 
RESULTADO(%)= %6.32


,ID,EXAMENES_por_concurso,correctores,exam_cada_uno,ganancia_cada_corrector,costo_total,costo_por_examen
19,19,64,2,64,30960.0,61920.0,967.500000
11,11,112,2,56,27840.0,55680.0,497.142857
16,16,120,2,60,29400.0,58800.0,490.000000
18,18,213,3,71,33690.0,101070.0,474.507042
14,14,237,3,79,36810.0,110430.0,465.949367
13,13,255,3,85,39150.0,117450.0,460.588235
15,15,270,3,90,41100.0,123300.0,456.666667
8,8,290,3,96,43440.0,130320.0,449.379310
10,10,292,3,97,43830.0,131490.0,450.308219
12,12,301,4,75,35250.0,141000.0,468.438538


In [8]:
piso_vec = [4000,6000,8000]
tope_vec = [35000,40000,45000,50000]
max_correccion_vec = [80,100,120]
divide_en_max_vec = [1,2]
correctores_inicial_vec = [2,3,4]
agregan_vec = [1,2]
vec_params = [piso_vec,tope_vec,max_correccion_vec,divide_en_max_vec,correctores_inicial_vec,agregan_vec]

# producto cartesiano
params = []
import itertools
for vec in itertools.product(*vec_params): # * unpackea la lista en distintos argumentos
    params.append(list(vec))
params = pd.DataFrame(params, columns=['piso','tope','max_correccion','divide_en_max','correctores_inicial','agregan'])
params['pend'] = round((params.tope-params.piso)/(params.max_correccion),0)
params['limite'] = limite

In [9]:
RESULTADOS = armado_simulacion(params, dist_año, 20)

In [10]:
RESULTADOS

,piso,tope,max_correccion,divide_en_max,correctores_inicial,agregan,pend,limite,FIJO20_costo_bootstrap,FIJO20_media_costo,...,MITAD_MAS_diferencia_limite_boot,MITAD_MAS_media_boot,MITAD_MAS_std_boot,MITAD_MAS_porcentaje_inferior_0,DOBLE_costo_bootstrap,DOBLE_media_costo,DOBLE_diferencia_limite_boot,DOBLE_media_boot,DOBLE_std_boot,DOBLE_porcentaje_inferior_0
0,4000,35000,80,1,2,1,388.0,3950000,"[3987344.0, 3220760.0, 4526404.0, 3277888.0, 3...",3721012.4,...,"[-1633128.0, -838848.0, -1517144.0, -1931804.0...",-1392061.4,463622.969197,1.0,"[6543388.0, 7513256.0, 7572592.0, 7454908.0, 6...",7262168.4,"[-2593388.0, -3563256.0, -3622592.0, -3504908....",-3312168.4,586995.710214,1.0
1,4000,35000,80,1,2,2,388.0,3950000,"[3425840.0, 3373368.0, 4046064.0, 3774336.0, 3...",3655341.6,...,"[-2222904.0, -2608112.0, -2342056.0, -1308448....",-1907399.6,516062.976675,1.0,"[6968744.0, 7852752.0, 7003960.0, 6345416.0, 7...",7742195.2,"[-3018744.0, -3902752.0, -3053960.0, -2395416....",-3792195.2,722625.911606,1.0
2,4000,35000,80,1,3,1,388.0,3950000,"[3305684.0, 3745804.0, 3935092.0, 3967804.0, 3...",3788621.0,...,"[-1586972.0, -1347780.0, -2415904.0, -1340316....",-1598775.6,662080.648577,1.0,"[7150132.0, 8049652.0, 6742936.0, 6699148.0, 6...",7226046.6,"[-3200132.0, -4099652.0, -2792936.0, -2749148....",-3276046.6,553304.950573,1.0
3,4000,35000,80,1,3,2,388.0,3950000,"[3724800.0, 4356572.0, 3669224.0, 3336248.0, 3...",3949756.8,...,"[-2155076.0, -1707012.0, -3244020.0, -116128.0...",-1953540.6,706302.764170,1.0,"[7712756.0, 10135064.0, 7675508.0, 6826328.0, ...",7819931.6,"[-3762756.0, -6185064.0, -3725508.0, -2876328....",-3869931.6,943250.118188,1.0
4,4000,35000,80,1,4,1,388.0,3950000,"[3682272.0, 3696628.0, 3790108.0, 3568496.0, 3...",3793355.6,...,"[-1025960.0, -2175028.0, -2504864.0, -1695332....",-1746646.2,588175.311191,1.0,"[7620956.0, 8092236.0, 7224504.0, 5809740.0, 6...",7350062.8,"[-3670956.0, -4142236.0, -3274504.0, -1859740....",-3400062.8,680412.455265,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,8000,50000,120,2,2,2,350.0,3950000,"[4009900.0, 3411500.0, 4986000.0, 5125000.0, 3...",3946875.0,...,"[-2413000.0, -1523400.0, -2870000.0, -1642800....",-2029180.0,484168.296773,1.0,"[8318800.0, 8141800.0, 8100700.0, 7442300.0, 6...",7529555.0,"[-4368800.0, -4191800.0, -4150700.0, -3492300....",-3579555.0,600412.676811,1.0
428,8000,50000,120,2,3,1,350.0,3950000,"[5197150.0, 4207200.0, 4353000.0, 4088600.0, 3...",4173485.0,...,"[-1646600.0, -2241700.0, -2021900.0, -1726200....",-1930277.5,286666.873511,1.0,"[7127800.0, 8090500.0, 7914650.0, 7706450.0, 7...",7907902.5,"[-3177800.0, -4140500.0, -3964650.0, -3756450....",-3957902.5,345742.945147,1.0
429,8000,50000,120,2,3,2,350.0,3950000,"[4309650.0, 4427750.0, 4038550.0, 3961400.0, 4...",4319292.5,...,"[-1530350.0, -2613450.0, -1891800.0, -2765500....",-2247212.5,543886.069613,1.0,"[9337350.0, 8179900.0, 8798200.0, 8318700.0, 8...",8295582.5,"[-5387350.0, -4229900.0, -4848200.0, -4368700....",-4345582.5,643378.385415,1.0
430,8000,50000,120,2,4,1,350.0,3950000,"[3613350.0, 2970800.0, 3394150.0, 3846400.0, 3...",3788840.0,...,"[-1272400.0, -2393900.0, -2434000.0, -1745800....",-1943985.0,416176.672550,1.0,"[7854800.0, 7797350.0, 8154500.0, 7832900.0, 8...",8053750.0,"[-3904800.0, -3847350.0, -4204500.0, -3882900....",-4103750.0,379942.676926,1.0


In [12]:
version = 1
params.to_csv(f'/Users/lucaspecina/Desktop/Planificacion Estrategica/Pagos jurados consejo magist/pruebas lineales/ACTUAL/2021_simulaciones_{version}.csv',index=False)


In [13]:
plotear_resultado(RESULTADOS,vec_params)

interactive(children=(Dropdown(description='tipo', options=('violin', 'hist'), value='violin'), Dropdown(descr…

<function __main__.plotear(tipo='violin', piso=6000, tope=40000, max_correccion=100, divide_en_max=1, correctores_inicial=2, agregan=1)>